In [11]:
import numpy as np
import pandas as pd
import optuna
from scipy.sparse import csr_matrix
from weighting_strategies import (
    bm25_weight, tfidf_weight, normalized_weight,
    log_weight, confidence_weight, power_weight,
    pmi_weight, robust_user_centric_weight, sigmoid_propensity_weight, power_lift_weight, robust_user_centric_weight_v2
)
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import train_test_split, precision_at_k, ndcg_at_k

import cornac


In [12]:
import sys
import os

# Add the parent directory to sys.path to resolve imports from sibling directories
sys.path.append(os.path.abspath(".."))

from utils.sparse import transform_dataframe_to_sparse

In [13]:
movielens_df = (
    pd.DataFrame(
        data=cornac.datasets.movielens.load_feedback(variant="20M"),
        columns=['user_id', 'item_id', 'target']
    )
    .loc[:, ['user_id', 'item_id', 'target']]
    .dropna()
)
movielens_df['user_id'].nunique(), movielens_df['item_id'].nunique(), movielens_df.shape[0]

(138493, 26744, 20000263)

In [18]:
user_item_matrix, user_mapping, item_mapping = transform_dataframe_to_sparse(
    movielens_df, row_field='user_id', col_field='item_id', data_field='target'
)


train_val_mat, test_mat = train_test_split(user_item_matrix, train_percentage=0.9, random_state=42)
train_mat, val_mat = train_test_split(train_val_mat, train_percentage=0.9, random_state=42)

print(f"Train Shape: {train_mat.shape}, Val Shape: {val_mat.shape}, Test Shape: {test_mat.shape}")

Train Shape: (138493, 26744), Val Shape: (138493, 26744), Test Shape: (138493, 26744)


In [ ]:
results_folder = "results/movielens_20m"
results_filename = "movielens_20m_experiment_results.csv"

import time

def run_hyperparameter_optimization(
    train_mat: csr_matrix,
    val_mat: csr_matrix,
    train_val_mat: csr_matrix,
    test_mat: csr_matrix,
    weighting_strategy: str,
    algorithm: str,
    n_trials: int = 20,
    output_dir: str = None,
) -> pd.DataFrame:
    results = []
    algorithms = {
        "ALS_factors=10": lambda: AlternatingLeastSquares(factors=10, random_state=42),
        "ALS_factors=100": lambda: AlternatingLeastSquares(factors=100, random_state=42),
    }
    strategies = [
        "no_weighting",
        "bm25",
        "tfidf",
        "log", 
        "confidence",
        "power",
        "normalized",
        "pmi",
        "robust_user_centric",
        "robust_user_centric_weight_v2",
        "sigmoid_propensity",
        "power_lift"
    ]
    if weighting_strategy not in strategies:
        raise ValueError(f"Weighting strategy '{weighting_strategy}' is not recognized.")
    strategy = weighting_strategy

    if algorithm not in algorithms:
        raise ValueError(f"Algorithm '{algorithm}' is not recognized.")
    algo_name = algorithm
    AlgoFactory = algorithms[algorithm]

    print(f"Running optimization for {algo_name} with {strategy}...")

    def get_weighted_matrix(matrix, params):
        weighted = matrix.copy()
        if strategy == "bm25":
            weighted = bm25_weight(weighted, K1=params.get("bm25_k1"), B=params.get("bm25_b"))
        elif strategy == "confidence":
            weighted = confidence_weight(weighted, alpha=params.get("conf_alpha"))
        elif strategy == "power":
            weighted = power_weight(weighted, p=params.get("power_p"))
        elif strategy == "tfidf":
            weighted = tfidf_weight(weighted)
        elif strategy == "log":
            weighted = log_weight(weighted)
        elif strategy == "normalized":
            weighted = normalized_weight(weighted)
        elif strategy == "pmi":
            weighted = pmi_weight(weighted)
        elif strategy == "robust_user_centric":
            weighted = robust_user_centric_weight(weighted, scale_factor=params.get("scale_factor"))
        elif strategy == "sigmoid_propensity":
            weighted = sigmoid_propensity_weight(weighted, p=params.get("p"), beta=params.get("beta"))
        elif strategy == "power_lift":
            weighted = power_lift_weight(weighted, p=params.get("p"))
        elif strategy == "robust_user_centric_weight_v2":
            weighted = robust_user_centric_weight_v2(weighted, lower_q=params.get("lower_q"), upper_q=params.get("upper_q"))
        return weighted

    def objective(trial):
        params = {}
        # Suggest weighting strategy parameters
        if strategy == "bm25":
            params["bm25_k1"] = trial.suggest_float("bm25_k1", 0.1, 1000)
            params["bm25_b"] = trial.suggest_float("bm25_b", 0.0, 1.0)
        elif strategy == "confidence":
            params["conf_alpha"] = trial.suggest_float("conf_alpha", 1.0, 150.0)
        elif strategy == "power":
            params["power_p"] = trial.suggest_float("power_p", 0.1, 1.5)
        elif strategy == "robust_user_centric":
            params["scale_factor"] = trial.suggest_float("scale_factor", 0.1, 10.0)
        elif strategy == "robust_user_centric_weight_v2":
            params["lower_q"] = trial.suggest_float("lower_q", 5.0, 45.0)
            params["upper_q"] = trial.suggest_float("upper_q", 55.0, 95.0)
        elif strategy == "sigmoid_propensity":
            params["p"] = trial.suggest_float("p", 0.1, 5.0)
            params["beta"] = trial.suggest_float("beta", 0.0, 1.0)
        elif strategy == "power_lift":
            params["p"] = trial.suggest_float("p", 0.1, 1.5)
        weighted_train = get_weighted_matrix(train_mat, params)
        
        # Train Model
        model = AlgoFactory()
        model.fit(weighted_train, show_progress=False)

        # Evaluate on Validation Set
        return ndcg_at_k(model, train_mat, val_mat, K=20, show_progress=False)

    # Optimize only if strategy has parameters
    current_trials = n_trials if strategy in ["bm25", "confidence", "power", "robust_user_centric", "robust_user_centric_weight_v2", "sigmoid_propensity", "power_lift"] else 1
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=current_trials, n_jobs=-1)

    # --- Final Retraining & Testing ---
    # Use best params to weight the full train_val matrix
    best_params = study.best_params
    weighted_train_val = get_weighted_matrix(train_val_mat, best_params)

    # Train Final Model
    final_model = AlgoFactory()
    
    start_time = time.time()
    final_model.fit(weighted_train_val, show_progress=False)
    end_time = time.time()
    
    # Evaluate on Test Set
    test_ndcg_10 = ndcg_at_k(final_model, train_val_mat, test_mat, K=10, show_progress=False)
    test_precision_10 = precision_at_k(final_model, train_val_mat, test_mat, K=10, show_progress=False)
    test_ndcg_20 = ndcg_at_k(final_model, train_val_mat, test_mat, K=20, show_progress=False)
    test_precision_20 = precision_at_k(final_model, train_val_mat, test_mat, K=20, show_progress=False)

    results.append({
        "Algorithm": algo_name,
        "Strategy": strategy,
        "Number of Optimization Trials": current_trials,
        "Best Val NDCG@20": study.best_value,
        "Test NDCG@10": test_ndcg_10,
        "Test NDCG@20": test_ndcg_20,
        "Test Precision@10": test_precision_10,
        "Test Precision@20": test_precision_20,
        "Final Train Time (s)": end_time - start_time,
        "Best Params": best_params
    })

    if output_dir:
        output_path = os.path.join(output_dir, f"{algo_name}_{strategy}_results.csv")
        pd.DataFrame(results).to_csv(output_path, index=False)
    return pd.DataFrame(results)

In [ ]:
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

# Execution of all experiments
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="confidence", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="confidence", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="ALS_factors=10", n_trials=20, output_dir=results_folder)
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="ALS_factors=100", n_trials=20, output_dir=results_folder)

[I 2026-01-29 21:47:01,417] A new study created in memory with name: no-name-95fa011a-625e-4a08-b999-5260cef9b94e


Running optimization for ALS_factors=10 with no_weighting...


[I 2026-01-29 21:47:05,852] Trial 0 finished with value: 0.2077362353361567 and parameters: {}. Best is trial 0 with value: 0.2077362353361567.
[I 2026-01-29 21:47:11,378] A new study created in memory with name: no-name-cd2f2006-c9d0-46b0-8413-1860250e32fa


Running optimization for ALS_factors=10 with bm25...


[I 2026-01-29 21:47:28,511] Trial 2 finished with value: 0.1993047029214265 and parameters: {'bm25_k1': 183.90837928197922, 'bm25_b': 0.15769347935927014}. Best is trial 2 with value: 0.1993047029214265.
[I 2026-01-29 21:47:28,536] Trial 3 finished with value: 0.20270874823184418 and parameters: {'bm25_k1': 358.4755935673906, 'bm25_b': 0.5242285788623163}. Best is trial 3 with value: 0.20270874823184418.
[I 2026-01-29 21:47:28,596] Trial 0 finished with value: 0.20292763343505626 and parameters: {'bm25_k1': 789.4936413291072, 'bm25_b': 0.6412531115079463}. Best is trial 0 with value: 0.20292763343505626.
[I 2026-01-29 21:47:28,634] Trial 1 finished with value: 0.2025130614083314 and parameters: {'bm25_k1': 679.7606494994739, 'bm25_b': 0.8189135357396745}. Best is trial 0 with value: 0.20292763343505626.
[I 2026-01-29 21:47:45,630] Trial 7 finished with value: 0.20008185011022017 and parameters: {'bm25_k1': 51.22215890719594, 'bm25_b': 0.18974311382739584}. Best is trial 0 with value: 0

...

Running optimization for ALS_factors=100 with sigmoid_propensity...


[I 2026-01-29 23:07:25,076] Trial 3 finished with value: 0.21303492480230518 and parameters: {'p': 4.272882114962067, 'beta': 0.1486848421398217}. Best is trial 3 with value: 0.21303492480230518.
[I 2026-01-29 23:07:25,088] Trial 0 finished with value: 0.2265921550847041 and parameters: {'p': 0.8071933903223973, 'beta': 0.30326212277823505}. Best is trial 0 with value: 0.2265921550847041.
[I 2026-01-29 23:07:25,134] Trial 1 finished with value: 0.24265763773674653 and parameters: {'p': 3.890162015753506, 'beta': 0.509360797138456}. Best is trial 1 with value: 0.24265763773674653.
[I 2026-01-29 23:07:25,150] Trial 2 finished with value: 0.23181043133613768 and parameters: {'p': 3.908972684779729, 'beta': 0.3425623305739033}. Best is trial 1 with value: 0.24265763773674653.
[I 2026-01-29 23:08:06,680] Trial 4 finished with value: 0.24479076634587388 and parameters: {'p': 3.865086212830957, 'beta': 0.5538473518454491}. Best is trial 4 with value: 0.24479076634587388.
[I 2026-01-29 23:08:0

Running optimization for ALS_factors=100 with power_lift...


[I 2026-01-29 23:11:06,397] Trial 3 finished with value: 0.24200520786631688 and parameters: {'p': 0.5961303364462469}. Best is trial 3 with value: 0.24200520786631688.
[I 2026-01-29 23:11:06,441] Trial 1 finished with value: 0.26479639419525575 and parameters: {'p': 0.3957683875015411}. Best is trial 1 with value: 0.26479639419525575.
[I 2026-01-29 23:11:06,458] Trial 0 finished with value: 0.1305932824975531 and parameters: {'p': 1.4062261136728622}. Best is trial 1 with value: 0.26479639419525575.
[I 2026-01-29 23:11:06,489] Trial 2 finished with value: 0.1580469378195184 and parameters: {'p': 1.1836541061029249}. Best is trial 1 with value: 0.26479639419525575.
[I 2026-01-29 23:11:47,947] Trial 4 finished with value: 0.234959350056575 and parameters: {'p': 0.1068195224966865}. Best is trial 1 with value: 0.26479639419525575.
[I 2026-01-29 23:11:48,011] Trial 7 finished with value: 0.1338665700129183 and parameters: {'p': 1.3788569661864043}. Best is trial 1 with value: 0.2647963941

,Algorithm,Strategy,Best Val NDCG@20,Test NDCG@20,Test Precision@20,Final Train Time (s),Best Params
23,ALS_factors=100,power_lift,0.265945,0.337178,0.368855,10.815068,{'p': 0.35035165551041647}
22,ALS_factors=100,sigmoid_propensity,0.259962,0.333955,0.366206,10.792080,"{'p': 1.5152466760045404, 'beta': 0.9959842331..."
12,ALS_factors=100,no_weighting,0.252795,0.324987,0.357838,10.774874,{}
19,ALS_factors=100,pmi,0.257309,0.323910,0.346753,10.830424,{}
20,ALS_factors=100,robust_user_centric,0.254878,0.322906,0.354965,10.681851,{'scale_factor': 9.729574967756513}
17,ALS_factors=100,power,0.254827,0.321246,0.353753,10.822451,{'power_p': 1.305569938241399}
14,ALS_factors=100,tfidf,0.256439,0.318898,0.349704,10.811064,{}
16,ALS_factors=100,confidence,0.255824,0.317208,0.347325,10.843457,{'conf_alpha': 1.281009213528539}
13,ALS_factors=100,bm25,0.249887,0.309938,0.344770,10.824066,"{'bm25_k1': 172.20254865698615, 'bm25_b': 0.33..."
15,ALS_factors=100,log,0.231924,0.307046,0.342517,10.828904,{}


In [ ]:
import glob

all_results = []
# Match any CSV in the result folder
for f in glob.glob(f"{results_folder}/*.csv"):
    all_results.append(pd.read_csv(f))

if all_results:
    experiment_results = pd.concat(all_results)
    experiment_results = experiment_results.sort_values("Test NDCG@20", ascending=False)
    experiment_results.to_csv(results_filename, index=False)
else:
    print("No results found.")

experiment_results